# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 9 2023 8:30AM,254489,10,Yusen-8998253,Yusen – 3D Matrix,Released
1,Jan 6 2023 3:07PM,254490,19,2022102425,"GUSA Granules USA, Inc.",Released
2,Jan 6 2023 2:48PM,254487,16,8998216,Sartorius Bioprocess Solutions,Released
3,Jan 6 2023 2:23PM,254485,19,ADV80010132,"AdvaGen Pharma, Ltd",Released
4,Jan 6 2023 2:23PM,254485,19,ADV80010133,"AdvaGen Pharma, Ltd",Released
5,Jan 6 2023 2:22PM,254484,10,CTF0011575,"Citieffe, Inc.",Released
6,Jan 6 2023 2:17PM,254483,19,ELI010923,Eli Lilly and Company,Released
7,Jan 6 2023 1:57PM,254479,19,ADV80010134,"AdvaGen Pharma, Ltd",Released
8,Jan 6 2023 1:57PM,254479,19,ADV80010135,"AdvaGen Pharma, Ltd",Released
9,Jan 6 2023 1:57PM,254479,19,ADV80010136,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,254484,Released,1
31,254485,Released,2
32,254487,Released,1
33,254489,Released,1
34,254490,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
254484,NaN,1.0
254485,NaN,2.0
254487,NaN,1.0
254489,NaN,1.0
254490,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
254406,2.0,1.0
254409,0.0,3.0
254417,0.0,1.0
254418,0.0,1.0
254419,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
254406,2,1
254409,0,3
254417,0,1
254418,0,1
254419,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,254406,2,1
1,254409,0,3
2,254417,0,1
3,254418,0,1
4,254419,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,254406,2,1
1,254409,,3
2,254417,,1
3,254418,,1
4,254419,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc."
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd"
5,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc."
6,Jan 6 2023 2:17PM,254483,19,Eli Lilly and Company
7,Jan 6 2023 1:57PM,254479,19,"AdvaGen Pharma, Ltd"
18,Jan 6 2023 1:56PM,254478,10,ISDIN Corporation
37,Jan 6 2023 1:56PM,254480,10,Emerginnova
40,Jan 6 2023 1:51PM,254476,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix,,1
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc.",,1
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions,,1
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd",,2
4,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc.",,1
5,Jan 6 2023 2:17PM,254483,19,Eli Lilly and Company,,1
6,Jan 6 2023 1:57PM,254479,19,"AdvaGen Pharma, Ltd",,11
7,Jan 6 2023 1:56PM,254478,10,ISDIN Corporation,,19
8,Jan 6 2023 1:56PM,254480,10,Emerginnova,1,2
9,Jan 6 2023 1:51PM,254476,10,ISDIN Corporation,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix,1,
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc.",1,
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions,1,
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd",2,
4,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc.",1,
5,Jan 6 2023 2:17PM,254483,19,Eli Lilly and Company,1,
6,Jan 6 2023 1:57PM,254479,19,"AdvaGen Pharma, Ltd",11,
7,Jan 6 2023 1:56PM,254478,10,ISDIN Corporation,19,
8,Jan 6 2023 1:56PM,254480,10,Emerginnova,2,1
9,Jan 6 2023 1:51PM,254476,10,ISDIN Corporation,3,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix,1,
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc.",1,
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions,1,
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd",2,
4,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix,1.0,NaN
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc.",1.0,NaN
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions,1.0,NaN
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd",2.0,NaN
4,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jan 9 2023 8:30AM,254489,10,Yusen – 3D Matrix,1.0,0.0
1,Jan 6 2023 3:07PM,254490,19,"GUSA Granules USA, Inc.",1.0,0.0
2,Jan 6 2023 2:48PM,254487,16,Sartorius Bioprocess Solutions,1.0,0.0
3,Jan 6 2023 2:23PM,254485,19,"AdvaGen Pharma, Ltd",2.0,0.0
4,Jan 6 2023 2:22PM,254484,10,"Citieffe, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2290190,129.0,1.0
12,254469,0.0,1.0
15,763332,3.0,4.0
16,1017741,4.0,0.0
18,508878,1.0,1.0
19,2544649,34.0,0.0
20,508910,2.0,0.0
21,254466,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2290190,129.0,1.0
1,12,254469,0.0,1.0
2,15,763332,3.0,4.0
3,16,1017741,4.0,0.0
4,18,508878,1.0,1.0
5,19,2544649,34.0,0.0
6,20,508910,2.0,0.0
7,21,254466,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,129.0,1.0
1,12,0.0,1.0
2,15,3.0,4.0
3,16,4.0,0.0
4,18,1.0,1.0
5,19,34.0,0.0
6,20,2.0,0.0
7,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,129.0
1,12,Released,0.0
2,15,Released,3.0
3,16,Released,4.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Executing,1.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0
Released,129.0,0.0,3.0,4.0,1.0,34.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Executing,1.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0
1,Released,129.0,0.0,3.0,4.0,1.0,34.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Executing,1.0,1.0,4.0,0.0,1.0,0.0,0.0,0.0
1,Released,129.0,0.0,3.0,4.0,1.0,34.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()